In [82]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import re
import itertools
from formatage_donnees import (
    dictionnaire_especes,
    nettoyer_nom_scientifique,
    extract_numbers,
    extraction_annee_mois_jour_dataframe,
    assigner_periode,
    completer_df,
)


In [141]:
#Formatage des données brutes en filtrant les colonnes d'intéret 

classes=['mammiferes', 'crustaces', 'escargots', 'fonges', 'reptiles', 'insectes','poissons','plantes','autres','oiseaux']
classes=['oiseaux']


for classe in classes:
    print(classe)
    # Chemin vers le fichier
    chemin_fichier = 'C:/Users/anormand/Documents/Projet Python/Biodiv/Data/raw_inpn/extractINPN_'+classe+'_26042024.csv'

    # Spécifiez le chemin du répertoire que vous souhaitez créer
    nouveau_repertoire = 'C:/Users/anormand/Documents/Projet Python/Biodiv/Data/filtered'

    # Vérifiez si le répertoire n'existe pas déjà
    if not os.path.exists(nouveau_repertoire):
        # Si le répertoire n'existe pas, créez-le
        os.makedirs(nouveau_repertoire)
        print("Répertoire créé avec succès à", nouveau_repertoire)
    else:        print("Le répertoire existe déjà à", nouveau_repertoire)

    # Liste des colonnes à importer
    colonnes_a_importer = [ 'codeInseeDepartement','nomScientifiqueRef','nomVernaculaire','regne', 'classe', 'ordre', 'famille', 'genre','especeProtegee','dateObservation','latitude','longitude','codeMaille10Km']
    
    # Lire uniquement les colonnes spécifiées du fichier dans un DataFrame
    df  = pd.read_csv(chemin_fichier, usecols=colonnes_a_importer,dtype=str)

    df = df.rename(columns={'nomScientifiqueRef': 'nomScientifique'})
    """
    # Application de la fonction à la colonne codeInseeDepartement pour créer la colonne temp
    df['Code'] = df['codeInseeDepartement'].apply(extract_numbers)

    # Supprimer les lignes avec temp vide ('')
    df = df[df['Code'] != '']

    # Sépare les valeurs multiples en listes
    df['Code'] = df['Code'].str.split(', ')

    # Utilise explode pour créer de nouvelles lignes
    df = df.explode('Code')
    df['Code']=df['Code'].astype(str)

    #ajouter une donnée groupe
    df['groupe'] = df['Code'].str.split(', ')
    """
    if classe=='reptiles':
        df['classe'] = df['classe'].fillna('Reptilia')

    # Ajouter des colonnes year, month, day et day_of_year
    df=extraction_annee_mois_jour_dataframe(df)
    
    
    print('nombre d"espèces observées :',len(df['nomScientifique'].unique()))
    print('nombre d"observations :',len(df))
    df.to_csv('C:/Users/anormand/Documents/Projet Python/Biodiv/Data/filtered/'+classe+'_filtered.csv', index=False)


oiseaux
Le répertoire existe déjà à C:/Users/anormand/Documents/Projet Python/Biodiv/Data/filtered
nombre d"espèces observées : 2617
nombre d"observations : 51734918


In [114]:
for col in df.columns:
    print(col)

idSINPOccTax
libelleCadreAcquisition
idCadreAcquisition
descriptionCadreAcquisition
objectifCadreAcquisition
motsClefsCadreAcquisition
referenceBiblioCadreAcquisition
maitreOuvrage
maitreOeuvre
financeur
contactPrincipal
typeFinancement
libelleJeuDonnees
idJeuDonnees
descriptionJeuDonnees
objectifJeuDonnees
jsonProtocole
libelleProtocole
idCampanuleProtocole
motsClefsJeuDonnees
territoireJeuDonnees
fournisseurJeuDonnees
producteurJeuDonnees
typeDonneesJeuDonnees
idOrigine
statutSource
statutObservation
observateur
determinateur
nomScientifique
nomCite
nomVernaculaire
cdNom
cdRef
rangTaxo
regne
classe
ordre
famille
genre
espece
groupeTaxoSimple
groupeTaxoAvance
dateObservation
datePrecision
decennie
annee
mois
dateDetermination
latitude
longitude
precisionGeometrieMetres
systemeCoordonnees
precisionLocalisation
typeObjetSource
toponyme
commune
codeInseeCommune
EPCI
codeInseeEPCI
departement
codeInseeDepartement
region
codeInseeRegion
dynamicProperties_natureObjetGeo
altitudeMin
atitudeM

In [142]:
#format 2 : regroupe les observations par espèces, par maille de 10km, sans données temporelles
classes= ['mammiferes', 'crustaces', 'escargots', 'fonges', 'reptiles', 'insectes','poissons','plantes','autres']
classes= ['oiseaux']
for classe in classes:
    
    print(classe)
    chemin = r'C:\Users\anormand\Documents\Projet Python\Biodiv\Data'
    nom_du_dossier ='/filtered'
    dossier=chemin+nom_du_dossier
    fichier=classe+'_filtered.csv'

    # Définir les colonnes à importer
    colonnes_a_importer = ['codeMaille10Km','nomVernaculaire','nomScientifique', 'genre', 'famille','ordre','classe','regne','especeProtegee','year']

    # Lire le fichier CSV
    df_raw = pd.read_csv(dossier + '/' + fichier, usecols=colonnes_a_importer,dtype={'codeMaille10Km': str})

    n_especes_entrée=len(df_raw['nomScientifique'].unique())
    n_obs_entrée=len(df_raw)
    print('En entrée : nombre d"espèces observées :',n_especes_entrée)
    print('En entrée : nombre d"obs :',n_obs_entrée)

    #nettoyage des données
    colonnes_obligatoires = ['nomScientifique','codeMaille10Km']
    df_cleaned = df_raw.dropna(subset=colonnes_obligatoires)
    df_cleaned = df_cleaned[df_cleaned['nomScientifique'].str.contains(' ')]
    #filtrer selon l'année d'observation
    df_cleaned= df_cleaned[df_cleaned['year']>=1970]
    
    df_cleaned['all']='All'
    df_cleaned['codeMaille10Km'] = df_cleaned['codeMaille10Km'].str.split('|').str[0].str.strip()

    df_cleaned['nomScientifique'] = df_cleaned['nomScientifique'].apply(nettoyer_nom_scientifique)
    
    """
    df_cleaned['altitudeMin']=df_cleaned['altitudeMin'].fillna('NA').astype(str)
    df_cleaned['nomVernaculaire']=df_cleaned['nomVernaculaire'].fillna('NA').astype(str)
    df_cleaned['nomScientifique']=df_cleaned['nomScientifique'].fillna('NA').astype(str)
    df_cleaned['regne']=df_cleaned['regne'].fillna('NA').astype(str)
    df_cleaned['classe']=df_cleaned['classe'].fillna('NA').astype(str)
    df_cleaned['ordre']=df['ordre'].fillna('NA').astype(str)
    df_cleaned['famille']=df_cleaned['famille'].fillna('NA').astype(str)
    df_cleaned['genre']=df_cleaned['genre'].fillna('NA').astype(str)
    """
    
    # Compter les occurrences d'observation de chaque taxon pour chaque code et période
    result = df_cleaned.groupby(['codeMaille10Km','nomScientifique']).size().reset_index(name='nombreObs')
    df_dico=dictionnaire_especes(df_cleaned)
    df_final=pd.merge(result,df_dico,on='nomScientifique')
    
    #Vérification de l'étape de nettoyage
    print(f'En sortie : nombre d"espèces observées :{len(df_final['nomScientifique'].unique())} soit une perte de {100-(round(len(df_final['nomScientifique'].unique())/n_especes_entrée*100))}%')
    print(f'En sortie : nombre d"obs :{result['nombreObs'].sum()} soit une perte de {100-(round(df_final['nombreObs'].sum()/n_obs_entrée*100))}%') 
    nombre_manquantes = df_raw['codeMaille10Km'].isna().sum()
    print(f"Nombre de données manquantes dans 'codeMaille10Km': {nombre_manquantes} sur un total de {len(df_raw)}, soit {round(nombre_manquantes/len(df_raw)*100)}%")

    #Enregistrer le fichier traité
    df_final.to_csv('C:/Users/anormand/Documents/Projet Python/Biodiv/Data/format2/data_INPN_1970_'+classe+'.csv', index=False)

oiseaux


C:\Users\anormand\AppData\Local\Temp\ipykernel_16516\1921815971.py:16: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(dossier + '/' + fichier, usecols=colonnes_a_importer,dtype={'codeMaille10Km': str})


En entrée : nombre d"espèces observées : 2617
En entrée : nombre d"obs : 51734918
En sortie : nombre d"espèces observées :1829 soit une perte de 30%
En sortie : nombre d"obs :44827179 soit une perte de 13%
Nombre de données manquantes dans 'codeMaille10Km': 6660323 sur un total de 51734918, soit 13%


In [145]:
df_dico

,nomVernaculaire,nomScientifique,genre,famille,ordre,classe,regne,all
0,Pinson des arbres,Fringilla coelebs,Fringilla,Fringillidae,Passeriformes,Aves,Animalia,All
1,Pouillot véloce,Phylloscopus collybita,Phylloscopus,Phylloscopidae,Passeriformes,Aves,Animalia,All
2,Mésange bleue,Cyanistes caeruleus,Cyanistes,Paridae,Passeriformes,Aves,Animalia,All
3,Canard colvert,Anas platyrhynchos,Anas,Anatidae,Anseriformes,Aves,Animalia,All
4,Verdier d'Europe,Chloris chloris,Chloris,Fringillidae,Passeriformes,Aves,Animalia,All
...,...,...,...,...,...,...,...,...
47963656,NaN,Antrostomus rufus,Antrostomus,Caprimulgidae,Caprimulgiformes,Aves,Animalia,All
48903527,Siffleur doré,Pachycephala chlorura littayei,Pachycephala,Pachycephalidae,Passeriformes,Aves,Animalia,All
48972961,NaN,Microbates collaris torquatus,Microbates,Polioptilidae,Passeriformes,Aves,Animalia,All
49628388,NaN,Recurvirostra americana,Recurvirostra,Recurvirostridae,Charadriiformes,Aves,Animalia,All
